# NBA Learning

![logo](http://iscasmc.ios.ac.cn/roll/lib/exe/fetch.php?media=wiki:logo.png)

This is a tutorial for the Java Library [```Regular Omega Language Learning (ROLL)```](http://iscasmc.ios.ac.cn/roll/doku.php) in a Groovy kernel Jupyter Notebook.
Groovy is very similar to Java and you can write all code in Java syntax.

**Tips** : If something goes strange, use the menu bar above ```Kernel -> Restart``` to reboot this notebook and run following code again.

---

**First of all, load the jar file of the learning library ROLL.**

In [1]:
%classpath add jar ROLL.jar

Added jar: [ROLL.jar]


In the active automata learning setting proposed by Angluin, there are a `teacher`, which knows the target language $L$, and a `learner`, whose task is to learn the target language, represented by an automaton, from the teacher by means of two kinds of queries: `membership queries` and `equivalence queries`. 
A membership query $MQ[w]$ asks whether a string $w$ belongs to $L$ while an equivalence query $EQ[A]$ asks whether the hypothesis automaton $A$ recognizes $L$. 
The teacher replies with a witness if the hypothesis is incorrect otherwise the learner completes its job.

In the following, we introduce two ways to learn the $\omega$-regular language  $L =\Sigma^* b^\omega$ over the alphabet $\Sigma = \{a, b\}$.
The first way is to use FDFA operations with the help of [DK package](http://www.brics.dk/automaton/) to play as the teacher.
The second way is to allow youself to play as the teacher.

**1. Learning the $\omega$-regular language L from DK by giving a target FDFA $F$ **

we first need to create the target FDFA $F$ which accepts the language $L$.

In [2]:
import roll.words.Alphabet
// you can always import all the classes in roll.jupyter package
import roll.jupyter.*
import java.util.List
import java.util.ArrayList

// in order to create an alphabet, you need an array of Characters
// the variable apList is local since there is type in front of it
List<Character> apList = new ArrayList<Character>();

// in Groovy, we have to do strong cast for Characters 
apList.add((char)'a');
apList.add((char)'b');

// create an alphabet with a Character list
// the created alphabet is global in this notebook
JupyterROLL.createAlphabet(apList);

// use JupyterROLL to create a leading DFA of F
// the variable target is global since there is no type in front of it
// so we can use this variable everywhere in this notebook
M = JupyterROLL.createDFA();

// now we can get the alphabet in the FDFA
alphabet = M.getAlphabet();

[0->a, 1->b]

In [3]:
import java.util.List
import java.util.ArrayList
import roll.jupyter.*
import roll.automata.DFA

// now we are ready to create a state in the learning DFA M
M.createState();

// 3 indices for the states
int fst = 0, snd = 1, thd = 2;
M.getState(fst).addTransition(alphabet.indexOf((char)'a'), fst); // 0 -> 0 via a
M.getState(fst).addTransition(alphabet.indexOf((char)'b'), fst); // 0 -> 0 via b

// now we create the progress DFA A for the state 0 in M
A = JupyterROLL.createDFA();

A.createState();
A.createState();
A.createState();

// the function getState is to get a state object by its state index
A.getState(fst).addTransition(alphabet.indexOf((char)'b'), snd); // 0 -> 1 via a
A.getState(fst).addTransition(alphabet.indexOf((char)'a'), thd); // 0 -> 2 via b
A.getState(snd).addTransition(alphabet.indexOf((char)'b'), snd); // 1 -> 1 via a
A.getState(snd).addTransition(alphabet.indexOf((char)'a'), thd); // 1 -> 2 via b
A.getState(thd).addTransition(alphabet.indexOf((char)'a'), thd); // 2 -> 2 via a
A.getState(thd).addTransition(alphabet.indexOf((char)'b'), thd); // 2 -> 2 via b

// set 0 as the initial state
A.setInitial(fst);
// set 2 as a final state
A.setFinal(snd);

List<DFA> proDFAs = new ArrayList<>();
proDFAs.add(A);

F = JupyterROLL.createFDFA(M, proDFAs);
// now we can output target in a DOT graph
F

Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 2->2 --> 2->2 a <!-- 2->2 --> 2->2 b <!-- 1->2 --> 1->2 a <!-- 1->1 --> 1->1 b <!-- 3 --> 3 <!-- 3->0 --> 3->0

Now we are ready to create an FDFA learner to learn a hypothesis FDFA $H$ from DK. 

We have 3 types of algorithms and 2 types of data structures to store the membership results during the learning process.

|                              | tree | table |
|:----------------------------:|:----------------------:|:-----------------------:|
| periodic     | ✔️                     | ✔️                      |
| syntactic | ✔️                     | ✔️                      |
| recurrent | ✔️                     | ✔️                      |

In the following, we will demostrate how to use the table-based recurrent learning algorithm to learn the target language $L$.

In [4]:
import roll.jupyter.*
    
// we create a global variable sequence which stores the learning procedure as a
// list of Triple object, the Triple object has three elements
// the first is the table/tree data structure, the second is the current hypothesis FDFA, and the third is the counterexample
// which refines the previous hypothesis FDFA to the current hypothesis
sequence = JupyterROLL.learningSeq("recurrent", "table", F);

// sequence is a java.util.List instance
sequence.size()

2

From the output of the learning list, the target language $L$ has been learned by the learning algorithm only with only 2 equivalence queries.

we now can check the Triple object at each step of the learning procedure

In [5]:
// initial learner data
sequence.get(0)

Learner 
 Hypothesis 
 Counterexample 
 
 
 Leading Learner: 
 || (ϵ, ϵ) | 
=============
ϵ || - | 
=============
a || - | 
b || - | 

Progress Learner for ϵ: 
 || ϵ | 
==============
ϵ || (+, -) | 
b || (+, +) | 
==============
a || (+, -) | 
ba || (+, -) | 
bb || (+, +) | 
 
 Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->0 --> 1->0 a <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0

In [6]:
// we get a new hypothesis after one counterexample refinement
sequence.get(1)

Learner 
 Hypothesis 
 Counterexample 
 
 
 Leading Learner: 
 || (ϵ, ϵ) | 
=============
ϵ || - | 
=============
a || - | 
b || - | 

Progress Learner for ϵ: 
 || ϵ | b | 
=======================
ϵ || (+, -) | (+, +) | 
b || (+, +) | (+, +) | 
a || (+, -) | (+, -) | 
=======================
ba || (+, -) | (+, -) | 
bb || (+, +) | (+, +) | 
aa || (+, -) | (+, -) | 
ab || (+, -) | (+, -) | 
 
 Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 2->2 --> 2->2 a <!-- 2->2 --> 2->2 b <!-- 1->2 --> 1->2 a <!-- 1->1 --> 1->1 b <!-- 3 --> 3 <!-- 3->0 --> 3->0 
 $ϵ(ab)^\omega$

**We have just learned how to learn the target language $L$ from DK. Sometimes we may not have the target FDFA in hand but we know exactly the language we want to learn in mind. ** 

In this case, we can first specify what kind of strings really belong to the target language $L$ and then refine the hypothesis if it does not recognize the target language by ourselves. 
We are going to use the tree-based recurrent learning algorithm to show how to learn the target language $L$ from ourselves. Note that for infinite words, we only care about the ultimately periodic words, which are the words represented by a finite prefix u and a periodic finite word v, i.e., $uv^\omega$.

**2. Learning the $\omega$-regular language $L = a^\omega + ab^\omega$ in an interactive way**

In [9]:
import roll.jupyter.*;
import java.util.function.BiFunction;
import roll.words.*;

// now we define a function :: (string, string) -> boolean and this function is used to 
// determine whether a string is in the target language
// this function resolves all membership queries posed by the learners
mqOracle = {
    // a^w + ab^w
    stem,loop -> 
    
    if (loop.length() < 1) return false;     // this is a finite word
    
    // check whether it is a^w 
    int numBInStem = 0, numBInLoop = 0;
    // counter the number of b's in stem
    for (int i = 0; i < stem.length(); i ++) {
        if(stem.charAt(i) == 'b') {
            numBInStem ++;
        }
    }
    // counter the number of b's in loop
    for (int i = 0; i < loop.length(); i ++) {
        if(loop.charAt(i) == 'b') {
            numBInLoop ++;
        }
    }
    
    if(numBInStem == 0 && numBInLoop == 0) return true;
    // check whether it is ab^w
    // check a's in loop
    if(numBInLoop != loop.length()) return false;
    if(stem.length() < 1) return false;
    if(stem.charAt(0) != 'a') return false;
    if(stem.length() - 1 != numBInStem) return false;
    return true;
};

// now we create a tree-based recurrent NBA learner to learn the target language L
fdfaLearner = JupyterROLL.createFDFALearner("recurrent", "table", mqOracle);

// we can also see the tree data structure of the learner in a DOT graph
fdfaLearner

Leading Learner: 
 || (ϵ, ϵ) | 
=============
ϵ || - | 
=============
a || - | 
b || - | 

Progress Learner for ϵ: 
 || ϵ | 
==============
ϵ || (+, -) | 
a || (+, +) | 
==============
b || (+, -) | 
aa || (+, +) | 
ab || (+, -) |

In [10]:
// output current hypothesis to see whether it recognizes the target language
fdfaLearner.getHypothesis()

Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 1->0 --> 1->0 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 <!-- 2->0 --> 2->0

In [11]:
// the hypothesis is not correct and we can use a counterexample (a, b)
// which is in the symmetric difference of the language of A and the target language
fdfaLearner.refineHypothesis("a", "b")
fdfaLearner.getHypothesis()

Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 1->0 --> 1->0 a <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 1: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->0 --> 1->0 a <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0

In [12]:
// hypothesis is still not correct, use (a,bab) to refine it
fdfaLearner.refineHypothesis("a", "bab")
fdfaLearner.getHypothesis()

Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 1->0 --> 1->0 a <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 1: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 2->2 --> 2->2 b <!-- 3 --> 3 3 <!-- 2->3 --> 2->3 a <!-- 1->2 --> 1->2 a <!-- 1->1 --> 1->1 b <!-- 3->2 --> 3->2 a <!-- 3->2 --> 3->2 b <!-- 4 --> 4 <!-- 4->0 --> 4->0

In [13]:
// hypothesis is still not correct, use (ba,b) to refine it
fdfaLearner.refineHypothesis("ba", "b")
fdfaLearner.getHypothesis()

Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 1->1 --> 1->1 b <!-- 1->2 --> 1->2 a <!-- 2->2 --> 2->2 a <!-- 2->2 --> 2->2 b <!-- 3 --> 3 <!-- 3->0 --> 3->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 1: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->0 --> 1->0 a <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0 Progress DFA for 2: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0

In [14]:
// hypothesis is still not correct, use (aa,a) to refine it
fdfaLearner.refineHypothesis("aa", "a")
fdfaLearner.getHypothesis()

Leading DFA M : <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 b <!-- 3 --> 3 3 <!-- 1->3 --> 1->3 a <!-- 4 --> 4 4 <!-- 1->4 --> 1->4 b <!-- 2->2 --> 2->2 a <!-- 2->2 --> 2->2 b <!-- 3->2 --> 3->2 b <!-- 3->3 --> 3->3 a <!-- 4->2 --> 4->2 a <!-- 4->4 --> 4->4 b <!-- 5 --> 5 <!-- 5->0 --> 5->0 Progress DFA for 0: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 1: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 2: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 0->0 --> 0->0 b <!-- 1 --> 1 <!-- 1->0 --> 1->0 Progress DFA for 3: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 b <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 a <!-- 1->0 --> 1->0 b <!-- 1->1 --> 1->1 a <!-- 2 --> 2 <!-- 2->0 --> 2->0 Progress DFA for 4: <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 a <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 b <!-- 1->0 --> 1->0 a <!-- 1->1 --> 1->1 b <!-- 2 --> 2 <!-- 2->0 --> 2->0

In [15]:
// hypothesis is correct, if we use (aa, aba) to refine it, the learner will report error message
fdfaLearner.refineHypothesis("aa", "aba")

Invalid counterexample, neither in hypothesis or target


null